<a href="https://colab.research.google.com/github/BuberDev/Machine_Learning/blob/main/MoE_v_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
!pip install transformers
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load the dataset
url = "http://ipfs.opentensor.ai/api/v0/object/get?arg=Qme2dawBzozFGtKWX73fh5fmB8NJD7TRS2XSWKhJB4WbJd"
response = requests.post(url)
text = response.content.decode('utf-8')

In [ ]:
# Tokenize the text
tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')
encoded_text = tokenizer.encode(text)

In [ ]:
input_ids = tokenizer.encode(encoded_text, return_tensors="pt")

In [ ]:
# Define the Mixture of Experts model
class MoEModel(nn.Module):
    def __init__(self, num_experts, expert_model):
        super(MoEModel, self).__init__()
        self.num_experts = num_experts
        self.experts = nn.ModuleList([expert_model() for i in range(num_experts)])
        self.gate = nn.Linear(1024, num_experts)
        
    def forward(self, input_ids):
        expert_outputs = []
        for i in range(self.num_experts):
            expert_outputs.append(self.experts[i](input_ids))
        expert_outputs = torch.stack(expert_outputs, dim=1)
        gate_outputs = self.gate(input_ids)
        gate_outputs = torch.softmax(gate_outputs, dim=1)
        mixed_output = torch.sum(expert_outputs * gate_outputs.unsqueeze(2), dim=1)
        return mixed_output

In [ ]:
! nvidia-smi

Tue Mar 21 14:45:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    33W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Initialize the MoE model
from transformers import GPTNeoConfig, GPTNeoForCausalLM, AutoTokenizer, AutoModel
!pip install moe

# Inicjalizacja konfiguracji modelu
#model_config = GPTNeoConfig.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Inicjalizacja modelu eksperta
#expert_model = GPTNeoForCausalLM(model_config)

#Alternatywny kod
expert_model_name = "EleutherAI/gpt-neo-1.3B"
expert_model = AutoModel.from_pretrained(expert_model_name)

# generowanie sekwencji z użyciem modelu MoEModel
tokenizer = AutoTokenizer.from_pretrained(expert_model_name)
inputs_embeds  = tokenizer(encoded_text, return_tensors="pt")["input_ids"]

# Inicjalizacja modelu MoE
num_experts = 4
model = MoEModel(num_experts, expert_model=expert_model, input_ids=inputs_embeds)

outputs = model(inputs_embeds=inputs_embeds)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of GPTNeoModel were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['transformer.h.7.attn.attention.bias', 'transformer.h.13.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'transformer.h.23.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.17.attn.attention.bias', 'transformer.h.3.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.5.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: ignored

W linii input_ids = tokenizer.encode(encoded_text, return_tensors="pt") powinno być text zamiast encoded_text, ponieważ encoded_text jest już zakodowanym tekstem.

W linii inputs_embeds = tokenizer(encoded_text, return_tensors="pt")["input_ids"] powinno być input_ids zamiast encoded_text.

W linii outputs = model(inputs_embeds=inputs_embeds) powinno być outputs = model(inputs_embeds) zamiast inputs_embeds=inputs_embeds.

W pętli treningowej powinno być inputs = input_ids[:, i:i+1024] zamiast inputs = torch.tensor(encoded_text[i:i+1024]).unsqueeze(0).

W pętli treningowej powinno być optimizer.zero_grad() przed loss.backward().

W pętli treningowej należy dodać total_loss += loss.item().

W pętli treningowej powinno być inputs[:, :-1] zamiast labels[:, :-1].

W pętli treningowej powinno być labels[:, 1:] zamiast labels[:, :-1].

Kod nie zawiera części kodu potrzebnej do uruchomienia modelu w trybie predykcji, dlatego kod ten jest tylko częściowo kompletny.